# Traffic analysis for Rzeszów


## Setup

In [ ]:
path_to_trafic_analysis_repo = ''

In [ ]:
!pip install osmnx networkx
!python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3

from google.colab import drive
drive.mount('/content/gdrive/')

import sys
sys.path.append(path_to_trafic_analysis_repo)


In [1]:
import random

import osmnx as ox
import networkx as nx
from multiprocessing import Pool
import itertools


import matplotlib.pyplot as plt
from typing import (
    List,
    Set,
    Tuple,
)

try:
    import importlib
    import src.graph_edit_functions
    import src.centrality_algorithms
    import src.plot_functions
    importlib.reload(src.graph_edit_functions)
    importlib.reload(src.plot_functions)
    importlib.reload(src.centrality_algorithms)
except Exception as e:
    print(e)

from src.graph_edit_functions import *
from src.centrality_algorithms import *
from src.plot_functions import *

No module named 'centrality_algorithms'


ModuleNotFoundError: No module named 'centrality_algorithms'

## Plot an unchanged graph of Rzeszow

In [ ]:
# Load the graph of Rzeszow

# bbox_rzeszow = (50.0647, 50.0154, 21.9618, 22.0454)
# g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow)

bbox_rzeszow_mini = (50.0383, 50.0535, 21.9748, 22.0153)
g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow_mini)

In [ ]:
plot_new_roads(g, [], 'rzeszow_unchanged')
unchanged_centralities = plot_centralities(g, f'rzeszow_unchanged');

## Remove Marszałkowska Street

In [ ]:
# Load the graph of Rzeszow

# bbox_rzeszow = (50.0647, 50.0154, 21.9618, 22.0454)
# g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow)

bbox_rzeszow_mini = (50.0383, 50.0535, 21.9748, 22.0153)
g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow_mini)
g_old = g.copy()

In [ ]:
# Remove nodes from Marszałkowska Street 

street_name='Marszałkowska'
removed_nodes = remove_street(g, edges, street_name=street_name)

plot_new_roads(g, removed_nodes, street_name)
removed_street_centralities = plot_centralities(g, f'rzeszow_removed_{street_name}')

In [ ]:
# Plot difference

plot_centralities_diff(g, unchanged_centralities, removed_street_centralities, f'rzeszow_removed_{street_name}')

## Add new connection between Rondo Jana Pawła II and Rondo Unii Europejskiej

In [ ]:
# Load the graph of Rzeszow

# bbox_rzeszow = (50.0647, 50.0154, 21.9618, 22.0454)
# g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow)

bbox_rzeszow_mini = (50.0383, 50.0535, 21.9748, 22.0153)
g, nodes, edges = get_graph_from_bbox(*bbox_rzeszow_mini)

In [ ]:
# Find nodes of target spots

nodes.reset_index(inplace=True)

jp2_north_south = 50.04454, 50.04605
jp2_west_east = 21.98106, 21.98465

y = nodes[nodes['y'].between(*jp2_north_south)]
x = nodes[nodes['x'].between(*jp2_west_east)]

found_nodes = pd.merge(x, y, how='inner')

print(f'Nodes for jp2: {found_nodes.osmid}')

ue_north_south = 50.04733, 50.04904
ue_west_east = 21.99830, 22.00050

y = nodes[nodes['y'].between(*ue_north_south)]
x = nodes[nodes['x'].between(*ue_west_east)]

found_nodes = pd.merge(x, y, how='inner')

print(f'Nodes for ue: {found_nodes.osmid}')

In [ ]:
# Add new connection

rondo_jp2_node_id = 270730841
rondo_ue_node_id = 34662452

add_edge(g, rondo_jp2_node_id, rondo_ue_node_id)
plot_new_roads(g, [(rondo_jp2_node_id, rondo_ue_node_id)], 'jp2-ue')
added_connection_centralities = plot_centralities(g, 'rzeszow_jp2_ue')

In [ ]:
# Plot difference

plot_centralities_diff(g, unchanged_centralities, added_connection_centralities, 'rzeszow_jp2_ue')